<a href="https://colab.research.google.com/github/leticiabbacellar/Python/blob/main/An%C3%A1lise_de_sentimento_elei%C3%A7%C3%B5es_EUA_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn
!pip install pandas
!pip install plotly
!pip install nltk
!pip install wordcloud
!python -m spacy download pt_core_news_sm
!pip install -q googletrans
!pip install -q deep_translator

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import plotly.express as px
import nltk
nltk.download('stopwords')
nltk.download('rslp')


from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
from collections import Counter
import string
import spacy
from nltk.stem import RSLPStemmer
from collections import defaultdict
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 91.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
df= pd.read_csv("/content/sample_data/sentiment_analysis.csv")
df

,Year,Month,Day,Time of Tweet,text,sentiment,Platform
0,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter
1,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook
2,2017,8,18,night,Don't angry me,negative,Facebook
3,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook
4,2022,6,8,noon,"Those who want to go, let them go",negative,Instagram
...,...,...,...,...,...,...,...
494,2015,10,18,night,"According to , a quarter of families under six...",negative,Twitter
495,2021,2,25,morning,the plan to not spend money is not going well,negative,Instagram
496,2022,5,30,noon,uploading all my bamboozle pictures of facebook,neutral,Facebook
497,2018,8,10,night,congratulations ! you guys finish a month ear...,positive,Twitter


In [3]:
def traduzir_p_portugues (text):
  try:
    return GoogleTranslator (source= "auto", target= "portuguese").translate(text)
  except Exception as e:
    print (f"Erro na tradução: {e}")

df["Comentário"] = df["text"].apply(traduzir_p_portugues)
df["Sentimento"]= df["sentiment"].map({
    "positive": "positivo",
    "negative": "negativo",
    "neutral": "neutro"
})
df["Horário"]= df["Time of Tweet"].map({
    "morning": "manhã",
    "noon": "tarde",
    "night": "noite"
})
df= df.rename(columns= {"Month": "Mês", "Year": "Ano", "Day": "Dia", "Platform": "Plataforma"})

df= df.drop(columns= ["text", "sentiment", "Time of Tweet"])
df.head(5)



,Ano,Mês,Dia,Plataforma,Comentário,Sentimento,Horário
0,2018,8,18,Twitter,Que grande dia !!! Parece o sonho.,positivo,manhã
1,2018,8,18,Facebook,"Sinto muito, sinto sua falta aqui na praia do mar",positivo,tarde
2,2017,8,18,Facebook,Não me brava,negativo,noite
3,2022,6,8,Facebook,Participamos da aula apenas para ouvir profess...,negativo,manhã
4,2022,6,8,Instagram,"Aqueles que querem ir, deixe -os ir",negativo,tarde


In [4]:
df.isnull().sum()

,0
Ano,0
Mês,0
Dia,0
Plataforma,0
Comentário,0
Sentimento,0
Horário,0


In [5]:
df.nunique()

,0
Ano,14
Mês,12
Dia,31
Plataforma,5
Comentário,393
Sentimento,3
Horário,3


In [6]:
print (df["Sentimento"].value_counts().sort_index())

Sentimento
negativo    134
neutro      199
positivo    166
Name: count, dtype: int64


In [7]:
print (df["Horário"].value_counts().sort_index())


Horário
manhã    171
noite    161
tarde    167
Name: count, dtype: int64


In [8]:
tabela= pd.crosstab(df["Horário"], df["Sentimento"])

tabela= pd.crosstab(df["Horário"], df["Sentimento"])

colormap={
    "positivo": "green",
    "negativo": "red",
    "neutro": "blue"
}

ordem = ["manhã", "tarde", "noite"]

fig= px.bar(
    tabela,
    barmode= "group",
    title= f"Distribuição das avaliações por sentimento e horário",
    labels = {"value": "Quantidade de comentários", "Horário": "Momento do dia"},
    color_discrete_map={str(k): v for k, v in colormap.items()},
    category_orders= {"Horário": ordem}
)
fig.show()


In [9]:
df_negativos= df[df["Sentimento"]== "negativo"]["Comentário"]
df_negativos.head(5)

from nltk.corpus import stopwords
stopwords= set(stopwords.words("portuguese"))

texto_unico_negativo=' '.join(df_negativos.dropna()).lower()
for caractere in string.punctuation:
  texto_unico_negativo= texto_unico_negativo.replace(caractere, "")
texto_unico_negativo

'não me brava participamos da aula apenas para ouvir professores lendo no slide apenas sem sentido aqueles que querem ir deixe os ir meu sharpie está correndo perigosamente baixo em tinta eu quero ir à música hoje à noite mas perdi minha voz não me distribua eu não me importo quem é você e o que você pode fazer tentei raspar meu pc e acessar o botão de reinicialização é realmente nojento sinto muito estou na vila agora e quase não encontrei um estagiário sentindo me sombrio se for um consolo fiz meu bmi testado hahaha diz que sou muito obsegui me muito por ficar infeliz por cerca de 10 minutos nascido e criado em nova york e morando no texas nos últimos 10 anos  eu ainda sinto falta de ny estou muuuuito egoa  o último dia da escola foi hoje  fungo  yor é tão não romântico estou tão entediado não gosto deste videoclipe deveria estar dormindo  perdi minha voz há alguns dias estou deprimido estou pensando em suicídio o que preciso fazer agora eu sou uma trepadeira que me sinto decepcionad

In [10]:
nlp = spacy.load("pt_core_news_sm")
doc= nlp(texto_unico_negativo)
doc

não me brava participamos da aula apenas para ouvir professores lendo no slide apenas sem sentido aqueles que querem ir deixe os ir meu sharpie está correndo perigosamente baixo em tinta eu quero ir à música hoje à noite mas perdi minha voz não me distribua eu não me importo quem é você e o que você pode fazer tentei raspar meu pc e acessar o botão de reinicialização é realmente nojento sinto muito estou na vila agora e quase não encontrei um estagiário sentindo me sombrio se for um consolo fiz meu bmi testado hahaha diz que sou muito obsegui me muito por ficar infeliz por cerca de 10 minutos nascido e criado em nova york e morando no texas nos últimos 10 anos  eu ainda sinto falta de ny estou muuuuito egoa  o último dia da escola foi hoje  fungo  yor é tão não romântico estou tão entediado não gosto deste videoclipe deveria estar dormindo  perdi minha voz há alguns dias estou deprimido estou pensando em suicídio o que preciso fazer agora eu sou uma trepadeira que me sinto decepcionada

In [11]:
filtrar= [
    token.lemma_ for token in doc
    if token.is_alpha and
    token.lemma not in stopwords and
    len(token.lemma_)>2 and
    token.pos_ in ['NOUN', 'VERB', 'ADJ']
]
filtrar

['brar',
 'participamo',
 'aula',
 'ouvir',
 'professor',
 'ler',
 'slide',
 'sentido',
 'querer',
 'deixar',
 'sharpie',
 'correr',
 'baixo',
 'tinta',
 'querer',
 'música',
 'noite',
 'perdi',
 'voz',
 'distribua',
 'importo',
 'poder',
 'fazer',
 'tentar',
 'raspar',
 'acessar',
 'botão',
 'reinicialização',
 'nojento',
 'sinto',
 'vila',
 'encontrar',
 'estagiário',
 'sentir',
 'sombrio',
 'consolo',
 'fiz',
 'bmi',
 'testar',
 'dizer',
 'obsegui',
 'ficar',
 'infeliz',
 'minuto',
 'nascer',
 'criar',
 'morar',
 'texa',
 'último',
 'ano',
 'sinto',
 'falta',
 'muuuuito',
 'egoa',
 'último',
 'dia',
 'escola',
 'fungo',
 'romântico',
 'entediar',
 'gostar',
 'videoclipe',
 'dever',
 'dormir',
 'perdi',
 'voz',
 'haver',
 'dia',
 'deprimido',
 'pensar',
 'suicídio',
 'preciso',
 'fazer',
 'trepadeira',
 'sinto',
 'decepcionar',
 'causa',
 'ter',
 'headch',
 'rount',
 'precisar',
 'fazer',
 'brar',
 'participamo',
 'aula',
 'ouvir',
 'professor',
 'ler',
 'slide',
 'sentido',
 'querer

In [12]:
top20_palavras= Counter (filtrar).most_common(20)
print("Top 20 palavras mais frequentes:")
for palavra, frequencia in top20_palavras:
  print(f"{palavra}: {frequencia}")

Top 20 palavras mais frequentes:
fazer: 13
poder: 10
sinto: 9
sentir: 9
ter: 9
falta: 8
querer: 7
dia: 7
saber: 7
noite: 5
fiz: 5
dormir: 5
casa: 5
levantar: 5
aula: 4
ouvir: 4
perdi: 4
tentar: 4
encontrar: 4
ficar: 4


In [13]:
fig= px.bar(
    x= [item [0] for item in top20_palavras],
    y=[item[1]for item in top20_palavras],
    labels= {"x": "Palavras mais comuns", "y":"Quantidade de vezes que aparecem"},
    text= [item [1] for item in top20_palavras]
)

fig.update_traces(marker_color="red", textposition= "inside")


fig.show()

In [14]:
stemmer= RSLPStemmer()

use_stem= [stemmer.stem(p) for p in filtrar]
use_stem

['br',
 'particip',
 'aul',
 'ouv',
 'profes',
 'ler',
 'slid',
 'sent',
 'quer',
 'deix',
 'sharpi',
 'corr',
 'baix',
 'tint',
 'quer',
 'músic',
 'noit',
 'perd',
 'voz',
 'distribu',
 'import',
 'pod',
 'faz',
 'tent',
 'rasp',
 'acess',
 'bot',
 'reinici',
 'nojent',
 'sint',
 'vil',
 'encontr',
 'estagi',
 'sent',
 'sombri',
 'consol',
 'fiz',
 'bmi',
 'test',
 'diz',
 'obsegu',
 'fic',
 'infeliz',
 'minut',
 'nasc',
 'cri',
 'mor',
 'tex',
 'últ',
 'ano',
 'sint',
 'falt',
 'muuuuit',
 'ego',
 'últ',
 'dia',
 'escol',
 'fung',
 'român',
 'entedi',
 'gost',
 'videoclip',
 'dev',
 'dorm',
 'perd',
 'voz',
 'hav',
 'dia',
 'deprim',
 'pens',
 'suicídi',
 'precis',
 'faz',
 'trep',
 'sint',
 'decepc',
 'caus',
 'ter',
 'headch',
 'rount',
 'precis',
 'faz',
 'br',
 'particip',
 'aul',
 'ouv',
 'profes',
 'ler',
 'slid',
 'sent',
 'quer',
 'deix',
 'sharpi',
 'corr',
 'baix',
 'tint',
 'quer',
 'músic',
 'noit',
 'perd',
 'voz',
 'distribu',
 'import',
 'pod',
 'faz',
 'tent',
 'rasp

In [21]:
#Organizando em uma função:

def analisar_por_plataforma (df, sentimento):
  df_plataforma= df[df["Sentimento"]== sentimento].copy()
  df_plataforma["Plataforma"] = df_plataforma["Plataforma"].str.strip()



  contagem = df_plataforma["Plataforma"].value_counts()
  df_plot = contagem.reset_index()
  df_plot.columns = ["Plataforma", "Quantidade"]
  print (f"Número de postagens em casa plataforma: {df_plot}")

  fig_= px.bar(
      df_plot,
      x= "Plataforma",
      y= "Quantidade",
      title= "Número de postagens de sentimento por plataforma",
      color= "Plataforma",
  )
  fig_.show ()
  return df_plataforma



In [22]:
df_avaliacoes_positivo = analisar_por_plataforma(df, "positivo")
df_avaliacoes_positivo

Número de postagens em casa plataforma: 48   Plataforma  Quantidade
0   Facebook          61
1  Instagram          57
2    Twitter          48


,Ano,Mês,Dia,Plataforma,Comentário,Sentimento,Horário
0,2018,8,18,Twitter,Que grande dia !!! Parece o sonho.,positivo,manhã
1,2018,8,18,Facebook,"Sinto muito, sinto sua falta aqui na praia do mar",positivo,tarde
6,2017,12,28,Facebook,2 da manhã feedings para o bebê são divertidas...,positivo,manhã
9,2018,5,28,Facebook,"Hoje, pela primeira vez, chego no barco. Sua j...",positivo,manhã
11,2016,4,10,Twitter,Eu realmente gosto da música Cardigan de Taylo...,positivo,noite
...,...,...,...,...,...,...,...
472,2019,11,2,Facebook,"250 milhas abaixo, apenas 1750 para ir. Tudo b...",positivo,tarde
474,2023,1,15,Twitter,Fil 461 com pessoas legais como eu,positivo,manhã
475,2023,1,18,Facebook,Bom dia!!! Trabalhar e então é o Baseball de d...,positivo,tarde
481,2021,9,15,Instagram,Por que beijar os pés das pessoas que o chutam...,positivo,tarde


In [17]:
df_avaliacoes_negativo = analisar_por_plataforma(df, "negativo")
df_avaliacoes_negativo

Número de postagens em casa plataforma:   Plataforma  Quantidade
0   Facebook          54
1  Instagram          49
2    Twitter          31


,Ano,Mês,Dia,Plataforma,Comentário,Sentimento,Horário
2,2017,8,18,Facebook,Não me brava,negativo,noite
3,2022,6,8,Facebook,Participamos da aula apenas para ouvir profess...,negativo,manhã
4,2022,6,8,Instagram,"Aqueles que querem ir, deixe -os ir",negativo,tarde
12,2016,5,15,Facebook,Meu Sharpie está correndo perigosamente baixo ...,negativo,manhã
13,2016,11,15,Instagram,"Eu quero ir à música hoje à noite, mas perdi m...",negativo,tarde
...,...,...,...,...,...,...,...
491,2023,1,24,Instagram,"Não, não é triste. Deve te deixar orgulhoso",negativo,noite
492,2023,1,27,Facebook,"Desculpe, vamos tentar mantê -lo baixo.",negativo,manhã
494,2015,10,18,Twitter,"Segundo, um quarto das famílias com menos de s...",negativo,noite
495,2021,2,25,Instagram,O plano de não gastar dinheiro não está indo bem,negativo,manhã
